# ADV Example

The following example shows a simple workflow for analyzing ADV data using DOLfYN's tools.

Start by importing the necessary DOLfYN tools:

In [1]:
# DOLfYN's core code (reading instrument data and rotating thru coordinate systems) has been migrated into MHKiT:
from mhkit import dolfyn as dlfn
# Analysis functionality hasn't yet
import dolfyn.adv.api as api

## Read Raw Instrument Data

DOLfYN currently only carries support for the Nortek Vector ADV. The example here loaded is simple 'tabletop' test data.

In [2]:
dat = dlfn.read_example('vector_data01.VEC') # use `dlfn.read([path to file + filename])` for your data
dat

Reading file c:\users\mcve343\mhkit-python\mhkit\..\examples\data\dolfyn\vector_data01.VEC ...
 end of file at 3000000 bytes.


<xarray.Dataset>
Dimensions:              (time: 122912, dir: 3, beam: 3, x*: 3, inst: 3, earth: 3)
Coordinates:
  * time                 (time) float64 1.34e+09 1.34e+09 ... 1.34e+09 1.34e+09
  * dir                  (dir) <U1 'X' 'Y' 'Z'
  * beam                 (beam) int32 1 2 3
  * x*                   (x*) int32 1 2 3
  * inst                 (inst) <U1 'X' 'Y' 'Z'
  * earth                (earth) <U1 'E' 'N' 'U'
Data variables:
    beam2inst_orientmat  (beam, x*) float64 2.709 -1.34 ... -0.3438 -0.3499
    c_sound              (time) float32 1.493e+03 1.493e+03 ... nan nan
    heading              (time) float32 5.6 10.5 10.51 10.52 ... nan nan nan nan
    pitch                (time) float32 -31.5 -31.7 -31.69 ... nan nan nan
    roll                 (time) float32 0.4 4.2 4.253 4.306 ... nan nan nan nan
    temp                 (time) float32 12.67 12.68 12.68 12.68 ... nan nan nan
    vel                  (dir, time) float32 -1.002 -1.008 -0.944 ... nan nan
    amp                  (dir, time) uint8 104 110 111 113 108 112 ... 0 0 0 0 0
    corr                 (dir, time) uint8 97 91 97 98 90 95 95 ... 0 0 0 0 0 0
    orientation_down     (time) bool True True True True ... True True True True
    pressure             (time) float64 5.448 5.436 5.484 5.448 ... 0.0 0.0 0.0
    orientmat            (inst, earth, time) float32 0.0832 0.155 ... -0.7065
Attributes:
    config:       {'ProLogID': 149, 'ProLogFWver': '4.08', 'config': 15412, '...
    inst_make:    Nortek
    inst_model:   Vector
    inst_type:    ADV
    rotate_vars:  ['vel']
    freq:         6000
    SerialNum:    VEC 9062
    Comments:     APL-UW vector on Tidal Turbulence Mooring in Admiralty, tim...
    fs:           32.0
    coord_sys:    inst
    has_imu:      0

The loaded dataset contains three primary groups of data. One: the velocity, amplitude, and correlation of the Doppler velocimetry; Two: measurements of the instrument's bearing and environment; and Three: the orientation matrices DOLfYN uses for rotating through coordinate frames.

## QC'ing Data

ADV velocity data tends to have spikes due to Doppler noise, and the common way to "despike" the data is by using the phase-space algorithm by Goring and Nikora (2002). DOLfYN integrates this function using a 2-step approach: create a logical mask where True corresponds to a spike detection, and then utilize an interpolation function to replace the spikes.

In [3]:
# Clean the file using the Goring+Nikora method:
mask = api.clean.GN2002(dat.vel, npt=5000)
# Replace bad datapoints via cubic spline interpolation
dat['vel'] = api.clean.clean_fill(dat['vel'], mask, npt=12, method='cubic')

print('Percent of data containing spikes: {0:.2f}%'.format(100*mask.mean()))

# If interpolation isn't desired:
dat_m = dat.copy(deep=True)
dat_m.coords['mask'] = (('dir','time'), ~mask)
dat_m['vel'] = dat_m.vel.where(dat_m.mask)

Percent of data containing spikes: 0.73%


## Coordinate Rotations

Now that the data has been cleaned, the next step is to rotate the velocity data into true East, North, Up coordinates

In [4]:
# First set the magnetic declination
dat_cln = dlfn.set_declination(dat, 10) # 10 degrees East

# Rotate that data from the instrument to earth frame:
dat_earth = dlfn.rotate2(dat_cln, 'earth')

Once in the true ENU frame of reference, we can calculate the principal flow direction for the velocity data and rotate it into principal coordinates (streamwise, cross-stream, vertical).

In [5]:
dat_earth.attrs['principal_heading'] = dlfn.calc_principal_heading(dat_earth.vel)
dat_fin = dlfn.rotate2(dat_earth, 'principal')

## Averaging Data

The next step in ADV analysis is to average the velocity data into time bins (ensembles). There are a couple ways to do this.

In [6]:
# Define an averaging object, create an 'ensembled' dataset and calculate basic turbulence statistics:
binner = api.ADVBinner(n_bin=9600, fs=dat_fin.fs, n_fft=4096)
dat_binned = binner(dat_fin)

# Alternatively, the functional version of ADVBinner:
dat_binned = api.calc_turbulence(dat_fin, n_bin=9600, fs=dat_fin.fs, n_fft=4096)

The benefit to using `ADVBinner` is that one has access to all of the velocity and turbulence analysis functions that DOLfYN contains. If basic analysis will suffice, the `calc_turbulence` function is the most convienent. Either option can still utilize DOLfYN's shortcuts.

See the DOLfYN API in the documentation for the full list of functions and shortcuts.

In [7]:
# 'do' functions return a full dataset:
dat_binned = binner.do_var(dat_fin, out_ds=dat_binned) # calculate the variance of each binned variable and add it to the averaged dataset

# 'calc' functions return a dataArray:
dat_binned['auto-covariance'] = binner.calc_acov(dat_fin['vel'])

# 'shortcuts' are accessed through the keyword 'Veldata' and can easily be added to the dataset
dat_binned['TI'] = dat_binned.Veldata.I

dat_binned

<xarray.Dataset>
Dimensions:               (time: 12, dir: 3, inst: 3, earth: 3, tke: 3, tau: 3, Sxx: 3, omega: 2048, dt: 2400)
Coordinates:
  * time                  (time) float64 1.34e+09 1.34e+09 ... 1.34e+09 1.34e+09
  * dir                   (dir) <U10 'streamwise' 'x-stream' 'vert'
  * inst                  (inst) <U1 'X' 'Y' 'Z'
  * earth                 (earth) <U1 'E' 'N' 'U'
  * tke                   (tke) <U5 'upup_' 'vpvp_' 'wpwp_'
  * tau                   (tau) <U5 'upvp_' 'upwp_' 'vpwp_'
  * Sxx                   (Sxx) <U3 'Suu' 'Svv' 'Sww'
  * omega                 (omega) float64 0.04909 0.09817 0.1473 ... 100.5 100.5
  * dt                    (dt) int32 0 1 2 3 4 5 ... 2395 2396 2397 2398 2399
Data variables: (12/28)
    U_std                 (time) float64 0.2618 0.282 0.07404 ... 1.522 1.542
    c_sound               (time) float32 1.492e+03 1.49e+03 ... 1.492e+03
    heading               (time) float32 80.63 79.42 37.51 ... 223.2 257.9 257.6
    pitch                 (time) float32 -31.96 -34.77 -39.39 ... -30.36 -30.19
    roll                  (time) float32 4.969 7.339 9.891 ... 1.156 35.7 35.61
    temp                  (time) float32 12.38 11.81 10.91 ... 11.01 11.52 12.26
    ...                    ...
    corr_var              (dir, time) float64 18.79 11.7 92.47 ... 309.1 249.3
    orientation_down_var  (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    pressure_var          (time) float64 9.36 96.0 ... 0.000348 0.0003839
    orientmat_var         (inst, earth, time) float64 0.03786 ... 2.109e-06
    auto-covariance       (dir, time, dt) float64 764.5 739.6 ... -1.068e+03
    TI                    (time) float64 0.5111 0.4365 0.1129 ... 2.134 3.212
Attributes: (12/16)
    inst_make:                 Nortek
    inst_model:                Vector
    inst_type:                 ADV
    rotate_vars:               ['vel']
    freq:                      6000
    SerialNum:                 VEC 9062
    ...                        ...
    declination:               10
    declination_in_orientmat:  1
    principal_heading:         45.2345
    n_bin:                     9600
    n_fft:                     4096
    n_fft_coh:                 1600